In [1]:
# -*- coding: utf-8 -*-

from __future__ import absolute_import

import cv2
import numpy as np
import os
import os.path as osp
import tensorflow as tf

from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Dropout, Softmax,Conv2DTranspose
from tensorflow.keras.layers import concatenate

def unit16b2uint8(img):
    
    if img.dtype == 'uint8':
        return img
    elif img.dtype == 'uint16':
        return img.astype(np.uint8)
    else:
        raise TypeError('No such of img transfer type: {} for img'.format(img.dtype))

###
def img_standardization(img):
    img = unit16b2uint8(img) #16位转8位
    
    if len(img.shape) == 2:
        img = np.expand_dims(img, 2) #增加维度
        img = np.tile(img, (1, 1, 1)) #扩充三倍 变为三通道
        return img
    elif len(img.shape) == 3:
        img = np.tile(img, (1, 1, 1)) #扩充三倍 变为三通道
        return img
    else:
        raise TypeError('The Depth of image large than 3 \n')

def bgr_to_gray(img):  #变为灰度图
    if len(img.shape)==3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        print('is rgb')
    return img 

def load_images(file_names,number):
    images = np.zeros((number,628,628,1),dtype=np.int8)
    i=0
    for file_name in file_names:
        img = cv2.imread(file_name, -1)
        img = img_standardization(img)
        img = img
        #img = resize(img, (640, 640), mode='constant', preserve_range=True)
        images[i] = img
        #print(images[i])
        i=i+1
    return images

def load_GT(file_names,number):
    images = np.zeros((number,628,628,1),dtype=np.uint8)
    #images = np.zeros((number,628,628,1))
    i=0
    for file_name in file_names:
        img = cv2.imread(file_name, -1)
        #img = unit16b2uint8(img)
        img = bgr_to_gray(img)
        img = img_standardization(img)
        img = (img > 0.5).astype(np.uint8)
        images[i] = img
        i=i+1
    return images


def save_image():
    
    return 

class BinaryThresholding:
    def __init__(self, threshold):
        self.threshold = threshold

    def __call__(self, img):
        gray = bgr_to_gray(img)
        (_, binary_mask) = cv2.threshold(gray, self.threshold, 255, cv2.THRESH_BINARY)
        binary_mask = cv2.medianBlur(binary_mask, 5)
        connectivity = 4
        _, label_img, _, _ = cv2.connectedComponentsWithStats(binary_mask , connectivity , cv2.CV_32S)
        return label_img
    

def load_data():
    #segmentor = BinaryThresholding(threshold=110) #阈值分割
    imagetrain_path = './dataset1/train/'
    resulttrain_path = './dataset1/train_GT/SEG'
    imagetest_path = './dataset1/test'
    #if not osp.exists(imagetrain_path):
    #    os.mkdir(resulttrain_path)
    
    train_image = [osp.join(imagetrain_path, image) for image in os.listdir(imagetrain_path)]
    train_images = load_images(train_image,175)
    print('done')
    train_result = [osp.join(resulttrain_path, image) for image in os.listdir(resulttrain_path)]
    train_results = load_GT(train_result,175)
    print('done')
    test_image = [osp.join(imagetest_path, image) for image in os.listdir(imagetest_path)]
    test_images = load_images(test_image,33)
    print('done')

    #train_images = np.array(tf.image.pad_to_bounding_box(train_images, offset_height=6, offset_width=6, target_height=640, target_width=640))

    #train_results = np.array(tf.image.pad_to_bounding_box(train_results, offset_height=6, offset_width=6, target_height=640, target_width=640))

    #test_images = np.array(tf.image.pad_to_bounding_box(test_images, offset_height=6, offset_width=6, target_height=640, target_width=640))

    print(train_images.shape)
    return train_images,train_results,test_images

image_train,label_train,image_test= load_data()


done
done
done
(175, 628, 628, 1)


In [2]:
## intersection over union
from tensorflow.keras import backend as K  
import os
os.environ['KERAS_BACKEND']='tensorflow'
def loss(y_true, y_pred):
    #y_pred = (y_pred > 0.5).astype(np.uint8)
    y_pred = tf.cast((y_pred>0.5),dtype=tf.int32)
    y_true = tf.cast(y_true,dtype=tf.int32)
    #y_pred_ = tf.to_int32(y_pred > 0.5)
    #score, up_opt = tf.keras.metrics.MeanIoU(y_true, y_pred, 2)
    m = tf.keras.metrics.MeanIoU(num_classes=2)
    _ = m.update_state(y_true, y_pred)
    score = m.result()
    #score, up_opt = tf.metrics.mean_iou
    return score
def dice_coef(y_true, y_pred, smooth=1): 
    y_true = tf.cast(y_true,dtype=tf.float32)
    intersection = K.sum(y_true * y_pred, axis=[1,2,3]) 
    union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3]) 
    return 1-K.mean( (2. * intersection + smooth) / (union + smooth), axis=0) 
def dice_coef_loss(y_true, y_pred): 
    1 - dice_coef(y_true, y_pred, smooth=1)
    
def IoU(y_true, y_pred, eps=1e-6):
    #if np.max(y_true) == 0.0:
    #    return IoU(1-y_true, 1-y_pred) ## empty image; calc IoU of zeros
    #y_pred = tf.cast((y_pred>0.5),dtype=tf.int32)
    y_true = tf.cast(y_true,dtype=tf.float32)
    intersection = K.sum(y_true * y_pred, axis=[1,2,3])
    union = K.sum(y_pred, axis=[1,2,3])
    #return 1-K.mean( (intersection) / (union), axis=0)
    return 1-((intersection) / (union))

def UNetModel(IMG_HEIGHT,IMG_WIDTH,IMG_CHANNELS):
    inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Dropout(0.2)(conv1)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D((2, 2))(conv1)
    #
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Dropout(0.2)(conv2)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D((2, 2))(conv2)
    #
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Dropout(0.2)(conv3)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)

    up1 = UpSampling2D(size=(2, 2))(conv3)
    up1 = concatenate([conv2,up1],axis=3)
    conv4 = Conv2D(64, (3, 3), activation='relu', padding='same')(up1)
    conv4 = Dropout(0.2)(conv4)
    conv4 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv4)
    #
    up2 = UpSampling2D(size=(2, 2))(conv4)
    up2 = concatenate([conv1,up2], axis=3)
    conv5 = Conv2D(32, (3, 3), activation='relu', padding='same')(up2)
    conv5 = Dropout(0.2)(conv5)
    conv5 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv5)
    #  注意，写成这种结构，并且用的loss为categorical_crossentropy，
    # 需要对groundtruth数据进行处理，见后面help_function.py里的mask_Unet
    conv6 = Conv2D(2, (1, 1), activation='relu',padding='same')(conv5)
    conv7 = Conv2D(1, (1, 1), activation='sigmoid')(conv6) 
    #conv6 = core.Reshape((2,IMG_WIDTH*IMG_HEIGHT))(conv6)
    #conv6 = core.Permute((2,1))(conv6)
    #conv7 = core.Activation('softmax')(conv6)
    model = Model(inputs=[inputs], outputs=[conv7])
    model.summary()
    model.compile(optimizer='sgd', loss=[dice_coef],metrics=['accuracy'])
    print('model compile')
    return model

#model = UNetModel(IMG_HEIGHT,IMG_WIDTH,IMG_CHANNELS)
#训练
model_name = 'modelnkhi.h5'
epochs = 16
batch_size = 1
validation_split = 0.1


model = UNetModel(628,628,1)
def model_fit(X_train,Y_train,model_name,epochs,batch_size,validation_split):
    earlystopper = EarlyStopping(patience=5, verbose=1)
    checkpointer = ModelCheckpoint(model_name, verbose=1, save_best_only=True)
    results = model.fit(X_train, Y_train, validation_split=validation_split,
                        batch_size=batch_size, epochs=epochs, 
                        callbacks =[ earlystopper,checkpointer],steps_per_epoch=150) 
earlystopper = EarlyStopping(patience=5, verbose=1)
checkpointer = ModelCheckpoint('modelnkhi.h5', monitor='loss', verbose=1, save_best_only=True)

#model_fit(image_train[:int(image_train.shape[0]*0.9)],label_train[:int(image_train.shape[0]*0.9)],model_name,epochs,batch_size,validation_split)
model_fit(image_train,label_train,model_name,epochs,batch_size,validation_split)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 628, 628, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 628, 628, 32) 320         input_1[0][0]                    
__________________________________________________________________________________________________
dropout (Dropout)               (None, 628, 628, 32) 0           conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 628, 628, 32) 9248        dropout[0][0]                    
______________________________________________________________________________________________

150/157 [===========================>..] - ETA: 0s - loss: 0.3547 - accuracy: 0.9371
Epoch 00010: val_loss did not improve from 0.36081
150/157 [===========================>..] - ETA: 0s - loss: 0.3698 - accuracy: 0.9356
Epoch 00011: val_loss did not improve from 0.36081
150/157 [===========================>..] - ETA: 0s - loss: 0.3574 - accuracy: 0.9348
Epoch 00012: val_loss did not improve from 0.36081
150/157 [===========================>..] - ETA: 0s - loss: 0.3405 - accuracy: 0.9356
Epoch 00013: val_loss improved from 0.36081 to 0.35863, saving model to modelnkhi.h5
150/157 [===========================>..] - ETA: 0s - loss: 0.3574 - accuracy: 0.9336
Epoch 00014: val_loss improved from 0.35863 to 0.35854, saving model to modelnkhi.h5
150/157 [===========================>..] - ETA: 0s - loss: 0.3328 - accuracy: 0.9383
Epoch 00015: val_loss did not improve from 0.35854
150/157 [===========================>..] - ETA: 0s - loss: 0.3470 - accuracy: 0.9363
Epoch 00016: val_loss improved 

In [3]:
def jaccord(image_TG,image_pred):
    jaccord_all=[]
    #preds_val = (preds_val > 0.5).astype(np.uint8)
    label_TG = np.unique(image_TG) #去掉重复数字，排序后输出
    label_pred = np.unique(image_pred)
    print(label_TG.shape,label_pred.shape)
    for lab_TG in label_TG:
        jaccord_max = 0
        visual_img = np.zeros(image_TG.shape)
        if lab_TG == 0.:
            continue
        for lab_pred in label_pred:
            if lab_pred == 0.:
                continue
            visual_img[image_TG==lab_TG]=1
            visual_img[image_pred==lab_pred]=visual_img[image_pred==lab_pred]+1
            and_j = sum(sum(sum(visual_img==2)))
            or_j = sum(sum(sum(visual_img==1)))
            jaccord_new = and_j/(and_j+or_j)
            if ((jaccord_new>0.5) &( jaccord_new>jaccord_max)):
                jaccord_max = jaccord_new
        jaccord_all.append(jaccord_max)
    jaccord = np.mean(jaccord_all)/len(label_TG)
    return jaccord

#from skimage.io import imread, imshow
#preds_val = model.predict(image_train[int(image_train.shape[0]*0.9):], verbose=1)   
#pred = np.zeros(preds_val.shape)
model = load_model('modelnkhi.h5',custom_objects={'dice_coef': dice_coef})
preds_val = model.predict(image_train[int(image_train.shape[0]*0.9):], verbose=1)


preds_val = preds_val[:,6:-6,6:-6,:]
print(preds_val)
preds_val = (preds_val > 0.5).astype(np.uint8)
preds_val = np.array(preds_val,np.uint8)
print(preds_val)
label_train = label_train[:,6:-6,6:-6,:]
label_train = np.array(label_train,np.uint8)
#print(label_train)
#_, preds_val, _, _ = cv2.connectedComponentsWithStats(preds_val)
jac = []
for i in range(preds_val.shape[0]):
    #imshow(preds_val[i,:])
    #plt.show(preds_val[i,:,:,0].all())
    _, preds_val0, _, _ = cv2.connectedComponentsWithStats(preds_val[i,:])
    _, preds_val1, _, _ = cv2.connectedComponentsWithStats(label_train[i,:])
    print(preds_val0)
    #preds_val[i,:,:,0] = preds_val0
    score = jaccord(preds_val1,preds_val0)
    print(score)
    jac.append(score)
print('平均评分',np.mean(jac))

18/18 [==============================] - 2s 92ms/sample
[[[[7.4246364e-06]
   [7.0300366e-06]
   [7.3272904e-06]
   ...
   [5.7469015e-06]
   [5.7384841e-06]
   [6.8734116e-06]]

  [[7.2109419e-06]
   [7.3367023e-06]
   [6.3222269e-06]
   ...
   [4.8983215e-06]
   [5.2190207e-06]
   [5.8077053e-06]]

  [[7.0510659e-06]
   [6.2780305e-06]
   [5.8998489e-06]
   ...
   [4.5824595e-06]
   [4.7304566e-06]
   [5.3980830e-06]]

  ...

  [[7.7544364e-06]
   [6.1701930e-06]
   [4.5737406e-06]
   ...
   [5.3815115e-06]
   [5.6677845e-06]
   [5.5160372e-06]]

  [[7.5335206e-06]
   [5.4156144e-06]
   [4.2124038e-06]
   ...
   [5.0316453e-06]
   [4.9101122e-06]
   [4.9730443e-06]]

  [[5.6859153e-06]
   [4.4864018e-06]
   [4.5277425e-06]
   ...
   [4.1708149e-06]
   [3.9902484e-06]
   [4.5287138e-06]]]


 [[[9.0454096e-06]
   [7.5665180e-06]
   [6.5819977e-06]
   ...
   [5.0048307e-06]
   [4.4165099e-06]
   [5.5489127e-06]]

  [[8.8534616e-06]
   [7.1927611e-06]
   [6.1359415e-06]
   ...
   [4.4814

TypeError: 'numpy.int32' object is not iterable